# Introduction to synthetic data workflow

![Standard workflow of generating and evaluating synthetic data with synthcity.](creatives/workflow.png)

The synthcity library captures the entire workflow of synthetic data generation and evaluation. The typical workflow contains the following steps, as illustrated above.

1. **Loading the dataset using a DataLoader**. The DataLoader class provides a consistent interface for loading and storing different types of input data (e.g. tabular, time series, and survival data). The user can also provide meta-data to inform downstream algorithms (e.g. specifying the sensitive columns for privacy-preserving algorithms).
2. **Training the generator using a Plugin**. In synthcity, the users instantiate, train, and apply different data generators via the Plugin class. Each Plugin represents a specific data generation algorithm. The generator can be trained using the fit() method of a Plugin.
3. **Generating synthetic data**. After the Plugin is trained, the user can use the generate() method to generate synthetic data. Some plugins also allow for conditional generation.
4. **Evaluating synthetic data**. Synthcity provides a large set of metrics for evaluating the fidelity, utility, and privacy of synthetic data. The Metrics class allows users to perform evaluation.

In addition, synthcity also has a Benchmark class that wraps around all the four steps, which is helpful for comparing and evaluating different generators.
After the synthetic data is evaluated, it can then be used in various downstream tasks.

# Case Study 1 - Data Modality
These notebooks are also available on Google Colab. This enables you to run the notebooks without having to set up an environment locally and gives you access to GPUs to run the notebooks on.

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1JOstMJmhI2wcufyBqZ1iV3YqOdThJ-_U?usp=sharing)

## 1. Introduction
![Catgorization of data modalities](creatives/data_modality.png)


\"Tabular data\" is a general category that encompasses many different data modalities. In this section, we introduce how to categorize these diverse modalities and how synthcity can be used to handle it.

### Single dataset

We start by introducing the most fundamental case where there is a single training dataset (e.g. a single DataFrame in Pandas). We characterize the data modalities by two axes: the observation pattern and the feature type.

The observation pattern describes whether and how the data are collected over time. There are three most prominent patterns, all supported by synthcity:

1. Static. All features are observed in a snapshot. There is no temporal ordering.
2. Regular time series.  Observations are made at regular intervals, t = 1, 2, 3... Of note, it is possible that different series may have different number of observations.
3. Irregular time series. Observations are made at irregular intervals, t = t1, t2, t3, ... Note that, for different series, the observation times may vary.

The feature type describes the domain of individual features. Synthcity supports the following three types. It also supports multivariate cases with a mixture of different feature types.

1. Continuous feature
2. Categorical feature
3. Integer feature
4. Censored feature: survival time and censoring indicator

The combination of observation patterns and feature types give rise to an array of data modalities. Synthcity supports all combinations.

### Composite dataset

A composite dataset involves multiple sub datasets. For instance, it may contain datasets collected from different sources or domains (e.g. from different countries). It may also contain both static and time series data. Such composite data are quite often seen in practice. For example, a patient's medical record may contain both static demographic information and longitudinal follow up data.

synthcity can handle the generation of different classes of composite datasets. Currently, it supports (1) multiple static datasets, (2) a static and a regular time series dataset, and (3) a static and a irregular time series dataset.

### Metadata

Very often we have access to metadata that describes the properties of the underlying data. Synthcity can make use of these information to guide the generation and evaluation process. It supports the following types of metadata:

1. sensitive features: indicator of sensitive features that should be protected for privacy.
2. outcome features: indicator of outcome feature that will be used as the  target in downstream prediction tasks.
3. domain: information about the data type and allowed value range.



### 1.1 The Task
In this first exercise, we will get used to loading datasets with the library and generating synthetic data from them, whatever the modality of the real data.

## 2. Imports
Lets get the imports out of the way. We import the required standard and 3rd party libraries and relevant Synthcity modules. We can also set the level of logging here, using Synthcity's bespoke logger. 

In [ ]:
# Standard
import sys
import warnings
from pathlib import Path

# 3rd party
import numpy as np
import pandas as pd

# synthcity
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import (GenericDataLoader, SurvivalAnalysisDataLoader, TimeSeriesDataLoader, TimeSeriesSurvivalDataLoader)

# Configure warnings and logging
warnings.filterwarnings("ignore")

# Set the level for the logging
# log.add(sink=sys.stderr, level="DEBUG")
log.remove()

## 3. Loading data of different modalities

In this notebook we will load different datasets into synthcity and show that data of many different modalities can be used to generate synthetic data using this module.

### 3.1 Static Data
Now we will start with the simplest example, static tabular data. For this, we will use the diabetes dataset from sklearn. First, we need to load the dataset.

In [ ]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True, as_frame=True)
X["target"] = y
display(X)

Then we pass it to the `GenericDataLoader` object from `synthcity`.

In [ ]:
loader = GenericDataLoader(
    X,
    target_column="target",
    sensitive_columns=["sex"],
)

We can print out different methods that are compatible with our data by calling `Plugins().list()` with a relevant list passed to the categories parameter.

In [ ]:
print(Plugins(categories=["generic"]).list())

No need to worry about the code in this next block here, we will go into lots of detail in how to generate synthetic data in the case studies to come. It is here purely to demonstrate that our dataset can be used to generate synthetic data using the synthcity module. We are using the method `marginal_distributions` to generate the synthetic data, which is one of the available debugging methods.

In [ ]:
syn_model = Plugins().get("marginal_distributions")
syn_model.fit(loader)
syn_model.generate(count=10).dataframe()

### 3.2 Static survival
Next lets look at censored data. Censoring is a form of missing data problem in which time to event is not observed for reasons such as termination of study before all recruited subjects have shown the event of interest or the subject has left the study prior to experiencing an event. Censoring is common in survival analysis. For our next example we will load a static survival dataset. Our dataset this time is a veteran lung cancer dataset provided by scikit-survival. 

First, load the dataset.

In [ ]:
from sksurv.datasets import load_veterans_lung_cancer

data_x, data_y = load_veterans_lung_cancer()
data_x["status"], data_x["survival_in_days"] = [record[0] for record in data_y], [record[1] for record in data_y]
display(data_x)

Pass it to the DataLoader. This time we will use the `SurvivalAnalysisDataLoader`. We need to pass it the data, the name of the column that contains our labels or targets to `target_column` and the the name of the column  containing the time elapsed when the event occurred (the event defined by the target column) to `time_to_event_column`. Calling `info()` on the loader object allows us to see the information about the dataset we have just prepared.

In [ ]:

loader = SurvivalAnalysisDataLoader(
    data_x,
    target_column="status",
    time_to_event_column="survival_in_days",
)
print(loader.info())


If we get the `marginal_distributions` plugin again and fit it to the `loader` object, we can then call `generate` to produce the synthetic data.

In [ ]:
syn_model = Plugins().get("marginal_distributions")
syn_model.fit(loader)
syn_model.generate(count=10)

### 3.3 Regular Time Series

In this next example we will load up a simple regular time series dataset and show that it is compatible with Synthcity. The temporal data must be passed to the loader as a list of DataFrames, where each DataFrame in the list refers to a different record and contains all time points for the record. So, there is a small amount of pre-processing to get our data into the right shape. As it is a regular time series we can simply pass a sequential list for each record.

The dataset we will use here is the basic motions dataset provided by SKTime. So, we need to import the library.

In [ ]:
from sktime.datasets import load_basic_motions

Load the data and re-format it into a list of DataFrames, where each DataFrame in the list refers to a different record and contains all time points for the record. We also need the outcomes as a DataFrame and the observation times as a list of time steps for each record. As this is a regular time series our time steps can simply be a sequential list of integers. We will also print the some of the data when we have it in the correct shape.

In [ ]:
X, y = load_basic_motions(
    split="TRAIN", return_X_y=True, return_type="pd-multiindex"
)
num_instances = len(set((x[0] for x in X.index)))
num_time_steps = len(set((x[1] for x in X.index)))

# Convert multi-index DataFrame into list of DataFrames
temporal_data = [X.loc[i] for i in range(num_instances)]
y = pd.DataFrame(y, columns=["label"])
observation_times = [list(range(num_time_steps)) for i in range(num_instances)]

print("The first 3 DataFrames in the list, `temporal_data`. They refer to the first 3 instances in the dataset. Each instance contains all time steps for all features.")
for i in range(3):
    display(temporal_data[i])
print("The first 3 label values, `y`.")
display(y[0:3])

Pass the data we just prepared to the DataLoader. Here we will use the `TimeSeriesDataLoader`. Then we will print out the loader info to check everything looks correct.

In [ ]:
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    outcome=y,
)
display(loader.dataframe())
print(loader.info())

Now we are ready to produce the synthetic data. We will use the `timegan` plugin to handle the time series data. As we don't care about the quality of the dataset here, we just want to check that it is compatible and practice loading datasets, we can pass `n_iter=1` to limit the number of iterations in the generator.

In [ ]:
syn_model = Plugins().get("timegan", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=10)

### 3.4 Irregular Time Series

Now lets load an irregular time series dataset and show that that is also compatible with Synthcity. The dataset we will use here is a google stocks dataset provided by the synthcity module itself.

In [ ]:
import numpy as np
from synthcity.utils.datasets.time_series.google_stocks import GoogleStocksDataloader

static_data, temporal_data, observation_times, outcome = GoogleStocksDataloader().load()

As the dataset is wrapped by synthcity, it is already provided to us in the correct format, but the requirements are the same as before. The temporal data is a list of DataFrames, where each DataFrame in the list refers to a different record and contains all time points for the record. The outcomes are all in one DataFrame and the observation times are a list of time steps for each record. The main difference here is that the observation times is a list of floats that represent the time between each data point.

In [ ]:
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    static_data=static_data,
    outcome=outcome,
)
print(loader.info())
display(loader.dataframe())

Exactly as for the regular time series, we can now generate synthetic data, by selecting our time series compatible plugin, then calling `fit()` and `generate()`.

In [ ]:
syn_model = Plugins().get("timegan", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=5)

### 3.5 Composite Irregular Time Series Survival Analysis

In this final example we will look at composite data while adding all the other more complex elements we have looked at so far. This next dataset is a composite irregular time series survival analysis dataset. 

Again this dataset is provided by synthcity, so there is little to do in terms of pre-processing as everything is in the right format to begin with.

In [ ]:
from synthcity.utils.datasets.time_series.pbc import PBCDataloader
(
    static_surv,
    temporal_surv,
    temporal_surv_horizons,
    outcome_surv,
) = PBCDataloader().load()
T, E = outcome_surv

print("The static survival features (`static_surv`) for the first 3 instances:")
display(static_surv[0:3])
print("The temporal survival features (`temporal_surv`) for the first 3 instances:")
for i in range(3):
    display(temporal_surv[i])
print("The observation times (`temporal_surv_horizons`) for the first 3 instances:")
display(temporal_surv_horizons[0:3])
print("The first 3 time to event values, `T`.")
display(T[0:3])
print("The first 3 event values, `E`.")
display(E[0:3])

Even complex datasets such as this are compatible with Synthcity. We can load this data using the `TimeSeriesSurvivalDataLoader`. Then by calling `loader.info()`, we can check the information about the dataset. It contains both one static feature ("sex") and 14 temporal features, making it a composite dataset. The `seq_time_id` field shows the irregular time sampling, which we create by passing the values to the `observation_times` parameter of the `TimeSeriesSurvivalDataLoader` object. And finally, we are formulating this data as a survival analysis problem, which is indicated by the presence of a `time_to_event` field.

In [ ]:
loader = TimeSeriesSurvivalDataLoader(
    temporal_data=temporal_surv,
    observation_times=temporal_surv_horizons,
    static_data=static_surv,
    T=T,
    E=E,
)

print(loader.info())

We can now generate synthetic data, in the way we are now well familiar with. We select our time series compatible plugin, then call `fit()` and `generate()`.

In [ ]:
syn_model = Plugins().get("timegan", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=5)

## 4. Extension

Use the code block below as a space to complete the extension exercises below.

### 4.1 Create synthetic datasets
 1) Above we have generated data with the debugging method `marginal_distributions` for tabular data and `timegan` for time series data. Now, using `Plugins().list()` or the [documentation](https://synthcity.readthedocs.io/en/latest/) find another method that is compatible with some of the datasets to see if you can generate your own synthetic data. What makes the method you have chosen better than the defaults we used before?
 
 2) Generate synthetic data for another dataset of your choice using the methods described above. You can use any of the other dataset from the sources we have used above: [SKLearn](https://scikit-learn.org/stable/datasets/toy_dataset.html), [SKTime](https://www.sktime.org/en/stable/api_reference/datasets.html), [SKSurv](https://scikit-survival.readthedocs.io/en/stable/api/datasets.html)  or [synthcity](https://github.com/vanderschaarlab/synthcity/tree/main/src/synthcity/utils/datasets) itself.

### 4.2 What is the best value for n_iter?
<details>
<summary>Show answer</summary>
It depends on your use case. The larger the value the longer it will take to run, however the plugins are equipped with early stopping, so that when a specified metric converges, the GAN stops at that point. So, setting an arbitrarily large value, is often a good option. 
</details>